In [1]:
#All the dependencies 
from bs4 import BeautifulSoup
import requests
from Resources.config import scrape_key
import pandas as pd
from datetime import date, datetime, timedelta

In [2]:
stonks = ['AMZN', 'AAPL', 'GOOGL', 'TSLA', 'GME']
stock_perf = []

#run through Scraping for each of the above symbols 

for stocks in stonks:
    query_url = f"https://api.scrapingdog.com/scrape?api_key={scrape_key}&url=https://finance.yahoo.com/quote/{stocks}?p={stocks}&.tsrc=fin-srch"
    r = requests.get(query_url).text
    soup = BeautifulSoup(r,'html.parser')
    alldata = soup.find_all("tbody")

#creating Tables to hold all the HTML values
    
    try:
     table1 = alldata[0].find_all("tr")
    except:
     table1 = None

    try:
     table2 = alldata[1].find_all('tr')
    except:
     table2 = None

    l={}
    u=list()
    
#Populating Lists & Dictionary with performance metrics

    for i in range(0,len(table1)):
     try:
       table1_td = table1[i].find_all('td')
     except:
       table1_td = None
     l[table1_td[0].text] = table1_td[1].text
     u.append(l)
     l={}

    for i in range(0,len(table2)):
     try:
       table2_td = table2[i].find_all('td')
     except:
       table2_td = None
     l[table2_td[0].text] = table2_td[1].text
     u.append(l)
     l={}

#Create Data Frame Rows
    
    scrape_df = pd.DataFrame(u)
    prev_close = float(scrape_df.iloc[0]['Previous Close'].replace(',',''))
    close_date = datetime.strptime((date.today() - timedelta(days = 1)).strftime('%m-%d-%Y'), '%m-%d-%Y')
    stock_list = [close_date, stocks, prev_close]
    stock_perf.append(stock_list)

#Final DataFrame Output

current_stock_data = pd.DataFrame(stock_perf, columns = ['Date', 'Symbol', 'Closing_Price'])
current_stock_data.head()

,Date,Symbol,Closing_Price
0,2020-12-04,AMZN,3186.73
1,2020-12-04,AAPL,122.94
2,2020-12-04,GOOGL,1821.84
3,2020-12-04,TSLA,593.38
4,2020-12-04,GME,16.12


In [3]:
current_stock_data['Date2'] = current_stock_data['Date'].dt.strftime('%m-%d-%Y')
final_current_stock_data = current_stock_data[['Date2', 'Symbol', 'Closing_Price']]
final_current_stock_data.rename(columns={'Date2': 'Date'}, inplace=True)
final_current_stock_data.head()

C:\Users\jorda\.conda\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Date,Symbol,Closing_Price
0,12-04-2020,AMZN,3186.73
1,12-04-2020,AAPL,122.94
2,12-04-2020,GOOGL,1821.84
3,12-04-2020,TSLA,593.38
4,12-04-2020,GME,16.12


In [4]:
from Resources.config import postgres_pwd
from sqlalchemy import create_engine


In [5]:
db_name = "KnightsDB3.0"

In [6]:
connection_string = f"postgres:{postgres_pwd}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# **Export to Postgres**
final_current_stock_data.to_sql(name="Web_Scraping_Data_Table", con=engine, if_exists='append', index=False)